# ResNet Finetuning
In the previous notebook, we trained a CNN from scratch and it performed poorly. In this notebook, we will use a pre-trained ResNet model and fine-tune it to our dataset. We will also use data augmentation to improve the model's performance.

### Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
from zipfile import ZipFile
import os
from shutil import copyfile
import shutil
from sklearn.model_selection import train_test_split

import torch
import torchvision
from torchvision.datasets import ImageFolder
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset, random_split
from torch.autograd import Variable
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR
from torchsummary import summary
from torchvision.transforms import RandomHorizontalFlip, RandomRotation, RandomVerticalFlip, ToTensor, Normalize, Resize, Compose, ColorJitter
from torchvision.utils import make_grid
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torchvision.models as models
import torch.optim as optim


from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

# Check for GPU availability
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


### Set up data folder and paths

In [2]:
data_folder = os.path.join(os.curdir, 'AllData')

train_path = os.path.join(data_folder, 'train')
test_path = os.path.join(data_folder, 'test')

### Data augmentation


In [3]:
train_transform = Compose([
    RandomHorizontalFlip(), # randomly flip and rotate
    Resize((300, 400)), # resize to 300 x 400
    RandomRotation(10), # randomly rotate by 10 degrees
    ToTensor(), # convert to tensor
    Normalize((0.5,), (0.5,)) # normalize the images
])

test_transform = Compose([
    Resize((300, 400)),
    ToTensor(),
    Normalize((0.5,), (0.5,))
])

In [4]:
train_data = ImageFolder(train_path, transform=train_transform)
test_data = ImageFolder(test_path, transform=test_transform)

print('Number of training images after augmentation: {}'.format(len(train_data)))
print('Number of testing images after augmentation: {}'.format(len(test_data)))

Number of training images after augmentation: 5484
Number of testing images after augmentation: 1378


In [5]:
batch_size=32

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

### Load the model and set parameters

In [6]:
model = models.resnet18(pretrained=True)

num_classes = 11
model.fc = nn.Linear(model.fc.in_features, num_classes)

optimizer = optim.Adam(model.parameters(), lr=1e-5, weight_decay=1e-5)
criterion = nn.CrossEntropyLoss()

scheduler = ReduceLROnPlateau(optimizer, 'min', patience=3)

model.to(DEVICE)

num_epochs = 20
best_val_acc = 0


/media/omar/4CC8FF56C8FF3D30/ArabWeatherTask/venv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/media/omar/4CC8FF56C8FF3D30/ArabWeatherTask/venv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [7]:
# Cross-validation
n_splits = 5  # Number of cross-validation folds
kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
fold = 0

val_accuracies = []
test_accuracies = []

In [8]:
for train_index, val_index in kf.split(range(len(train_data)), [label for (_, label) in train_data]):
    fold += 1
    print(f'Fold {fold}/{n_splits}')

    # Split data into train and validation sets for this fold
    train_fold, val_fold = random_split(
        train_data, [len(train_index), len(val_index)], generator=torch.Generator().manual_seed(42)  
    )

    # Create data loaders for this fold
    train_loader = DataLoader(train_fold, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_fold, batch_size=batch_size, shuffle=True)

    best_val_accuracy = 0.0
    early_stopping_patience = 5
    epochs_without_improvement = 0
    best_model_weights = None

    for epoch in range(num_epochs):
        model.train()
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        scheduler.step(loss)
            
    
        # Calculate validation 
        model.eval()
        correct_val= 0
        total_val = 0
        
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total_val += labels.size(0)
                correct_val += (predicted == labels).sum().item()
        
        val_accuracy = correct_val / total_val * 100
        print(f'Fold {fold}, Epoch {epoch + 1}/{num_epochs}, Validation Accuracy: {val_accuracy:.2f}%')
        
        # Check for early stopping
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            early_stopping_counter = 0
            best_model_weights = model.state_dict()
        else:
            early_stopping_counter += 1
            if early_stopping_counter >= early_stopping_patience:
                print("Early stopping triggered.")
                break

    # Load best model weights
    model.load_state_dict(best_model_weights)

    # Evaluate on the test set for this fold
    model.eval()
    correct_test = 0
    total_test = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total_test += labels.size(0)
            correct_test += (predicted == labels).sum().item()
    test_accuracy = correct_test / total_test * 100
    print(f'Fold {fold}, Test Accuracy: {test_accuracy:.2f}%')

    val_accuracies.append(val_accuracy)
    test_accuracies.append(test_accuracy)


# Calculate and print average validation and test accuracies over all folds
avg_val_accuracy = sum(val_accuracies) / n_splits
avg_test_accuracy = sum(test_accuracies) / n_splits

print(f'Average validation accuracy: {avg_val_accuracy:.2f}%')
print(f'Average test accuracy: {avg_test_accuracy:.2f}%')
    
        

Fold 1/5
Fold 1, Epoch 1/20, Validation Accuracy: 80.67%
Fold 1, Epoch 2/20, Validation Accuracy: 86.87%
Fold 1, Epoch 3/20, Validation Accuracy: 89.15%
Fold 1, Epoch 4/20, Validation Accuracy: 90.06%
Fold 1, Epoch 5/20, Validation Accuracy: 90.98%
Fold 1, Epoch 6/20, Validation Accuracy: 90.88%
Fold 1, Epoch 7/20, Validation Accuracy: 91.80%
Fold 1, Epoch 8/20, Validation Accuracy: 91.16%
Fold 1, Epoch 9/20, Validation Accuracy: 91.80%
Fold 1, Epoch 10/20, Validation Accuracy: 92.25%
Fold 1, Epoch 11/20, Validation Accuracy: 92.25%
Fold 1, Epoch 12/20, Validation Accuracy: 92.07%
Fold 1, Epoch 13/20, Validation Accuracy: 92.25%
Fold 1, Epoch 14/20, Validation Accuracy: 91.61%
Fold 1, Epoch 15/20, Validation Accuracy: 91.80%
Early stopping triggered.
Fold 1, Test Accuracy: 93.40%
Fold 2/5
Fold 2, Epoch 1/20, Validation Accuracy: 92.16%
Fold 2, Epoch 2/20, Validation Accuracy: 92.07%
Fold 2, Epoch 3/20, Validation Accuracy: 91.98%
Fold 2, Epoch 4/20, Validation Accuracy: 92.25%
Fold 2, 

In [ ]:
# Save model
torch.save(model.state_dict(), 'resnet18_finetuned.pth')

# Conclusion

In this notebook, we have explored two different implementations of image classification using PyTorch. Image classification is a fundamental task in computer vision, where the goal is to categorize images into predefined classes or categories. We have used the popular deep learning framework PyTorch to build and train convolutional neural network (CNN) models for this task.

## Code 1: Dataset Preparation and CNN Model Training

In the first code implementation, we focused on the following key steps:

1. **Data Preparation**:
   - Unzipped and organized the dataset.
   - Split the dataset into training, validation, and testing sets.
   - Loaded and preprocessed the images using PyTorch's `ImageFolder` and applied data augmentation techniques.

2. **Model Architecture**:
   - Defined a custom CNN architecture for image classification.

3. **Training and Evaluation**:
   - Trained the model using a stratified k-fold cross-validation approach.
   - Monitored and visualized training progress.
   - Evaluated the model's performance on a separate test set.

4. **Results Analysis**:
   - Calculated and displayed average validation and test accuracies.
   - Visualized sample images and class distributions.

5. **Model Saving**:
   - Saved the trained model for future use or deployment.

## Code 2: Transfer Learning with ResNet18

In the second code implementation, we used transfer learning with a pre-trained ResNet18 model:

1. **Data Preparation**:
   - Prepared the dataset in the same way as in the first code.

2. **Model Selection**:
   - Loaded a pre-trained ResNet18 model and adapted it for our classification task by modifying the fully connected layer.

3. **Training and Evaluation**:
   - Trained the modified model on the training dataset.
   - Evaluated the model on the validation and test sets.

4. **Learning Rate Scheduling**:
   - Implemented a learning rate scheduler to dynamically adjust the learning rate during training.

5. **Model Saving and Testing**:
   - Saved the best-performing model's weights and tested it on the test dataset.

Both implementations demonstrate different approaches to image classification, with the first code showcasing a custom CNN architecture and extensive data preprocessing, and the second code leveraging transfer learning with a pre-trained model to achieve impressive results.

We will go with the fine-tuned ResNet for the deployment of our model.
